In [ ]:
import datetime
from ib_async import *

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

times_list = []
prices_list = []
times_list_executed = []
prices_list_executed = []

try:
    contract = Stock('TSLA', 'SMART', 'USD')
    ib.qualifyContracts(contract)
    
    ticker = ib.reqMktData(contract, '', False, False)
    
    print(f"Waiting for data for {contract.symbol}...")
    ib.sleep(2) 
    
    if ticker.last:
        prices_list.append(ticker.last)
        times_list.append(datetime.datetime.now().isoformat())
        print(f"Latest real-time price for {contract.symbol}: **${ticker.last}**")
        contract = Stock('TSLA', 'SMART', 'USD')
      
        ib.qualifyContracts(contract)
        limit_price = ticker.last 
        order = LimitOrder('SELL', 11, limit_price)
        trade = ib.placeOrder(contract, order)
        print("issued order")
        while not trade.isDone():
            print("sleeping")
            ib.sleep(0.25)
        
        if trade.orderStatus.status == 'Filled':
            print(f"Success! {trade.contract.symbol} filled at {trade.orderStatus.avgFillPrice}")
            times_list_executed.append(datetime.datetime.now().isoformat())
            prices_list_executed.append(trade.orderStatus.avgFillPrice)
        else:
            print(f"Order ended with status: {trade.orderStatus.status}")
    else:
        print("Could not retrieve price. Check connectivity and market data subscriptions.")

finally:
    ib.cancelMktData(contract)
    ib.disconnect()

